In [1]:
import torch
import torchvision
from model import U2NETP
import torch.utils.mobile_optimizer as mobile_optimizer
from torch.utils.mobile_optimizer import optimize_for_mobile

In [2]:
print(torch.__version__)

1.7.0


In [6]:
!export CUDA_VISIBLE_DEVICES=0

In [7]:
# model_dir = '/home/xkaple00/JUPYTER_SHARED/Digis/Background_removal/U-2-Net_original/saved_models/u2netp/u2netp_bce_itr_4000_train_0.217968_tar_0.017879.pth'
# model_dir = '/home/xkaple00/JUPYTER_SHARED/Digis/Background_removal/U-2-Net_original/saved_models/u2netp/itr_3_train_0.177936_tar_0.014358.pth'
# model_dir = '/home/xkaple00/JUPYTER_SHARED/Digis/Background_removal/U-2-Net_original/saved_models/u2netp/itr_165000_train_28.574915_tar_0.008570_val_27.413172_val_tar_0.024370.pth'

model_dir = '/home/xkaple00/JUPYTER_SHARED/Digis/Background_removal/U-2-Net_original/saved_models/u2netp/batch15/best_itr_161000_train_2.904742_tar_0.009679_val_3.129026_val_tar_0.036401.pth'

In [13]:
model = U2NETP(3,1)
# model = torch.load_statedict(model_dir)
# model = model.load_state_dict(torch.load(model_dir))
# model = torch.load(model_dir)

In [14]:
model.load_state_dict(torch.load(model_dir))

<All keys matched successfully>

In [15]:
device = "cpu"
# data = data.to(device)
model.to(device)

U2NETP(
  (stage1): RSU7(
    (rebnconvin): REBNCONV(
      (conv_s1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (rebnconv1): REBNCONV(
      (conv_s1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv2): REBNCONV(
      (conv_s1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv3): REBNCONV(
      (conv_s1): Conv2d(16, 16, 

In [16]:
model.eval()

U2NETP(
  (stage1): RSU7(
    (rebnconvin): REBNCONV(
      (conv_s1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (rebnconv1): REBNCONV(
      (conv_s1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv2): REBNCONV(
      (conv_s1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv3): REBNCONV(
      (conv_s1): Conv2d(16, 16, 

In [17]:
example = torch.rand(1, 3, 320, 320)

In [18]:
traced_script_module = torch.jit.trace(model, example)

/home/xkaple00/anaconda3/envs/transformers/lib/python3.6/site-packages/torch/nn/functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/xkaple00/anaconda3/envs/transformers/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/xkaple00/anaconda3/envs/transformers/lib/python3.6/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [19]:
traced_script_module.save("Android_batch15.pt")

In [20]:
torchscript_model_optimized = optimize_for_mobile(traced_script_module)

In [21]:
torchscript_model_optimized.save("iOS_batch15_optimized.pt")

# Scripted models

In [22]:
scripted_script_module = torch.jit.script(model)

In [23]:
scripted_script_module.save("scripted_batch10.pt")

In [25]:
torchscript_model_optimized = optimize_for_mobile(scripted_script_module)

In [26]:
torchscript_model_optimized.save("scripted_batch10_optimized.pt")